# Purge Un-Verified Images

### Ref:  camera-api/example_utils.py  
(which came from ssd-dag)  

1. use labelImg to verify
2. move xml to annotation dir
3. move jpg to jpeg_images dir
4. OPTIONAL - make tf records
5. this program will read annotation
  - move unverified annotation to trash/
  - move unverified jpeg to trash/
  
 

In [8]:
import os
import sys
import shutil

In [9]:
# append to path so it wil find the tensorflow utilities

cwd = os.getcwd()
HOME = os.path.expanduser('~')
models = os.path.abspath(os.path.join(HOME, 'projects', 'tensorflow/models/research/'))
slim = os.path.abspath(os.path.join(  HOME, 'projects', 'tensorflow/models/research/slim'))
sys.path.append(models)
sys.path.append(slim)


import gen_imagesets
import example_utils

In [10]:
## Global
DATA_DATE = '20210124'

# project directories
PROJECT = os.getcwd()
HSDATA = '/hsdata'     # high speed data

IMAGE_DIR = os.path.join(HSDATA, "jpeg_images")
ANNOTATION_DIR_ROOT = os.path.join(HSDATA, "annotation")
TRASH_DIR = os.path.join(PROJECT, "trash")
TRAINING_SPLIT_TUPLE =  (100,0,0)


In [11]:
print ("annotation root:", ANNOTATION_DIR_ROOT)
verified_list, not_verified_list = gen_imagesets.gen_verified_lists(ANNOTATION_DIR_ROOT)
print ("verified list length:", len(verified_list))
print ("non-verified list length:", len(not_verified_list))

annotation root: /hsdata/annotation
-- MAKING Verified & NonVerified IMAGES LISTS
   found 12 subdirectories
   making list of verified annotations -- verified count 17894 / non 0 in /hsdata/annotation/202007
   making list of verified annotations -- verified count 10934 / non 0 in /hsdata/annotation/202002
   making list of verified annotations -- verified count 17953 / non 0 in /hsdata/annotation/202009
   making list of verified annotations -- verified count 11301 / non 0 in /hsdata/annotation/202001
   making list of verified annotations -- verified count 18768 / non 0 in /hsdata/annotation/202004
   making list of verified annotations -- verified count 18309 / non 0 in /hsdata/annotation/202008
   making list of verified annotations -- verified count 914 / non 33371 in /hsdata/annotation/202102
   making list of verified annotations -- verified count 19596 / non 0 in /hsdata/annotation/202101
   making list of verified annotations -- verified count 19473 / non 0 in /hsdata/annotat

In [12]:
# -- make sure you hvae a trash directory
#    camera-api/trash

# move unverfied to trash
error_count = 0

for i, image_id in enumerate(not_verified_list):
    if image_id == '.gitkeep':
        continue
    if i % 1000 == 0:
        print (f'moved -- {i}')
    try:
        xml_source, jpg_source = example_utils.get_image_annotation_paths(image_id)
        shutil.move(xml_source, TRASH_DIR)
        shutil.move(jpg_source, TRASH_DIR)
    except:
        error_count = error_count + 1
        print (f'could not move {xml_source} {jpg_source}')
        if error_count > 5:
            break
    
print (f'finished - moved {i}')


moved -- 0
moved -- 1000
moved -- 2000
moved -- 3000
moved -- 4000
moved -- 5000
moved -- 6000
moved -- 7000
moved -- 8000
moved -- 9000
moved -- 10000
moved -- 11000
moved -- 12000
moved -- 13000
moved -- 14000
moved -- 15000
moved -- 16000
moved -- 17000
moved -- 18000
moved -- 19000
moved -- 20000
moved -- 21000
moved -- 22000
moved -- 23000
moved -- 24000
moved -- 25000
moved -- 26000
moved -- 27000
moved -- 28000
moved -- 29000
moved -- 30000
moved -- 31000
moved -- 32000
moved -- 33000
finished - moved 33370


In [13]:
! rm -rf trash
! mkdir trash

# Backup to S3

In [ ]:
tarball_name = DATA_DATE + "_annotation.tar.gz"
annotation_source = os.path.join(HSDATA, 'annotation')
! tar czvf $tarball_name $annotation_source

In [ ]:
! aws s3 cp $tarball_name s3://jmduff.security-system/training_data/ --profile=jmduff

In [ ]:
tarball_name = DATA_DATE + "_jpeg_images.tar.gz"
jpeg_images_source = os.path.join(HSDATA, 'jpeg_images')
! tar cf - $jpeg_images_source | pigz > $tarball_name

In [ ]:
! aws s3 cp $tarball_name s3://jmduff.security-system/training_data/ --profile=jmduff